In [2]:
from gensim.models.fasttext import FastText

In [3]:
# .ft to .vec

# 파인튜닝 끝난 모델 경로
ft_path = "../data/cc.ko.300.ssafy.ft"
vec_path = "../data/cc.ko.300.ssafy_v1.vec"

model = FastText.load(ft_path)
model.wv.save_word2vec_format(vec_path, binary=False)

print("저장 완료:", vec_path)

저장 완료: ../data/cc.ko.300.ssafy_v1.vec


In [4]:
from pathlib import Path
import unicodedata
import re

In [5]:
# ko.dic 파싱

def is_hangul(text: str) -> bool:
    return bool(re.match(r'^[\u3130-\u318F\uAC00-\uD7A3]+$', text))

ko_dic_path = Path("../data/ko.dic")

dic_words = set()

with ko_dic_path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # '단어/품사' 형태라서 슬래시 앞부분만 사용
        word = line.split('/', 1)[0].strip()
        if not word:
            continue
        word = unicodedata.normalize("NFC", word)
        dic_words.add(word)

print("ko.dic 단어 수:", len(dic_words))

ko.dic 단어 수: 99791


In [6]:
# whitelist 단어 추가

whitelist_path = Path("../data/whitelist.txt")

whitelist = set()
with whitelist_path.open("r", encoding="utf-8") as f:
    for line in f:
        w = line.strip()
        if not w:
            continue
        w = unicodedata.normalize("NFC", w)
        whitelist.add(w)

print("화이트리스트 개수:", len(whitelist))

화이트리스트 개수: 68


In [7]:
# vec에 남길 단어 추출

orig_vec_path = Path("../data/cc.ko.300.ssafy_v1.vec")  # 파인튜닝 끝난 .vec 파일
out_vec_path  = Path("../data/cc.ko.300.ssafy_v1.clean.vec")

# 1) 첫 패스: 남길 단어 개수 세기
with orig_vec_path.open("r", encoding="utf-8", errors="ignore") as fin:
    header = fin.readline().strip()
    total_words, dim = map(int, header.split())
    print("원본 벡터 단어 수:", total_words, "차원:", dim)

    kept_count = 0
    for line in fin:
        line = line.rstrip()
        if not line:
            continue
        parts = line.split(" ")
        if len(parts) <= 2:
            continue

        raw_word = parts[0]
        word = unicodedata.normalize("NFC", raw_word)

        # 조건: (한글 + ko.dic) OR (화이트리스트)
        if (is_hangul(word) and word in dic_words) or (word in whitelist):
            kept_count += 1

print("남길 단어 개수:", kept_count)

# 2) 두 번째 패스: 실제로 새 .vec 파일 쓰기
with orig_vec_path.open("r", encoding="utf-8", errors="ignore") as fin, \
     out_vec_path.open("w", encoding="utf-8") as fout:

    _ = fin.readline()  # 원본 헤더 버림
    # 새 헤더: kept_count와 dim
    fout.write(f"{kept_count} {dim}\n")

    for line in fin:
        line = line.rstrip()
        if not line:
            continue
        parts = line.split(" ")
        if len(parts) <= 2:
            continue

        raw_word = parts[0]
        word = unicodedata.normalize("NFC", raw_word)

        if (is_hangul(word) and word in dic_words) or (word in whitelist):
            # 단어 + 나머지 벡터값 그대로 써줌
            fout.write(word + " " + " ".join(parts[1:]) + "\n")

print("완료! 새 .vec 파일:", out_vec_path)

원본 벡터 단어 수: 2000085 차원: 300
남길 단어 개수: 61754
완료! 새 .vec 파일: ..\data\cc.ko.300.ssafy_v1.clean.vec


In [ ]:
## Test

In [8]:
from gensim.models import KeyedVectors
from pathlib import Path

In [9]:
out_vec_path  = Path("../data/cc.ko.300.ssafy_v1.clean.vec")
kv = KeyedVectors.load_word2vec_format(str(out_vec_path), binary=False)
print("남은 단어 수:", len(kv.key_to_index))
print("벡터 차원:", kv.vector_size)

for word in ["싸피", "SSAFY", "에듀싸피", "월말평가", "입실설문", "취업", "알고리즘"]:
    if word in kv:
        print(f"\n[{word}] top 10:")
        for w, sim in kv.most_similar(word, topn=10):
            print(f"  {w:15s}  {sim:.4f}")
    else:
        print(f"\n[{word}]는 새 .vec에 없음")

남은 단어 수: 61751
벡터 차원: 300

[싸피] top 10:
  싸탈               0.9979
  멋사               0.9896
  우테코              0.9874
  SWEA             0.9859
  월말평가             0.9681
  프로젝트             0.9612
  관통프로젝트           0.9558
  싸피깃              0.9252
  퇴실체크             0.9157
  프로그래머스           0.9068

[SSAFY]는 새 .vec에 없음

[에듀싸피] top 10:
  우테코              0.8949
  SWEA             0.8818
  싸피               0.8756
  싸탈               0.8740
  월말평가             0.8704
  관통프로젝트           0.8663
  멋사               0.8639
  자율프로젝트           0.8576
  특화프로젝트           0.8538
  퇴실체크             0.8533

[월말평가] top 10:
  싸피               0.9681
  싸탈               0.9663
  우테코              0.9632
  멋사               0.9591
  SWEA             0.9575
  프로젝트             0.9361
  관통프로젝트           0.9327
  퇴실체크             0.9093
  싸피깃              0.9017
  프로그래머스           0.8980

[입실설문] top 10:
  우테코              0.8733
  싸피               0.8705
  싸탈               0.8659
  SWEA             0.8608
  월말평가 

C:\Users\owner\anaconda3\envs\ssafy-corpus\lib\site-packages\gensim\models\keyedvectors.py:851: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


In [11]:
for word in whitelist:
    if word in kv:
        print(f"\n[{word}] top 10:")
        for w, sim in kv.most_similar(word, topn=20):
            print(f"  {w:15s}  {sim:.4f}")
    else:
        print(f"\n[{word}]는 새 .vec에 없음")


[퇴실체크] top 10:
  우테코              0.9170
  싸피               0.9157
  SWEA             0.9122
  싸탈               0.9110
  월말평가             0.9093
  멋사               0.9079
  관통프로젝트           0.8997
  프로젝트             0.8846
  자율프로젝트           0.8687
  특화프로젝트           0.8618
  에듀싸피             0.8533
  프로그래머스           0.8494
  싸피깃              0.8472
  입실설문             0.8388
  출결소명서            0.8375
  퇴실설문             0.8269
  싸피이북             0.8261
  커뮤니케이션           0.8184
  잡싸피              0.8091
  과목평가             0.7998

[에듀싸피] top 10:
  우테코              0.8949
  SWEA             0.8818
  싸피               0.8756
  싸탈               0.8740
  월말평가             0.8704
  관통프로젝트           0.8663
  멋사               0.8639
  자율프로젝트           0.8576
  특화프로젝트           0.8538
  퇴실체크             0.8533
  프로그래머스           0.8479
  입실설문             0.8476
  퇴실설문             0.8432
  프로젝트             0.8422
  출결소명서            0.8376
  사유조퇴             0.8321
  싸피깃              0.8298
  잡싸피 